Michael Ricardo DS 2500 Project Trading torch

In [25]:
from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

In [26]:
def fetch_ticker_data(ticker, years_ago= 5):
    """
    Gets the market data for a given date and ticker.
    Fetches from yfinance library.

    Args:
        years ago (int): Representing the most recent day for stock entries 
        ticker(str): Representing the offical company stock ticker
    Returns
        df (DataFrame): DataFrame with Stock pricing data and history data
    """
    most_recent = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
    five_years_ago = (datetime.today() - timedelta(days= years_ago * 365)).strftime('%Y-%m-%d') 
    
    
    stock_data = yf.Ticker(ticker)
    returning_data = stock_data.history(start = five_years_ago, end= most_recent)

    return returning_data

In [27]:
#Data for the Disney
ticker = 'AAPL'
dis_data = fetch_ticker_data(ticker)
dis_data = dis_data.drop(columns=['Open', 'Dividends', 'Stock Splits'])
dis_data.index = dis_data.index.tz_convert('UTC')

In [28]:
#Data for the S&P 500 
ticker = '^GSPC'
SP500_data = fetch_ticker_data(ticker)
SP500_data = SP500_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits', 'Volume'])
SP500_data.index = SP500_data.index.tz_convert('UTC')

Adding variables not related to security itself: 

In [29]:
#Data for the S&P 500 
ticker = '^GSPC'
SP500_data = fetch_ticker_data(ticker)
SP500_data = SP500_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits', 'Volume'])
SP500_data.index = SP500_data.index.tz_convert('UTC')

In [30]:
#CBOE Volatility Index (VIX)
ticker = '^VIX'
VIX_data = fetch_ticker_data(ticker)
VIX_data = VIX_data.drop(columns=['Open', 'High', 'Low', 'Dividends', 'Stock Splits', 'Volume'])
VIX_data.index = VIX_data.index.tz_convert("UTC")

Section will be for data curration and preperation for regression 

In [31]:
def security_editor(df):
    """
    Gaining volume, volatility, and moving index metrics from the security in question. Averae True Range represents volatility for an asset over a weeks span/
    Simple moving average measures the average closing price for a week & month of a security. On-Balance volume takes into account volume movement within a seccurity.  

    Args:
        df (dataframe): Specific secuity df imported with Close, Volume, High, and Low data for each day of market activity 
    Returns
        df(dataframe): Added ATR. SMA_7, SMA_30, and OBV cols for given securuity. Removing high and low cols
    """
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=7)
    df = df.drop(columns=['High', 'Low'])
 
    df['SMA_7'] = ta.sma(df['Close'], length=10) 
    df['SMA_30'] = ta.sma(df['Close'], length=50)

    df['OBV'] = ta.obv(df['Close'], df['Volume'])
    return df

In [32]:
dis_refined = security_editor(dis_data)
print(dis_refined)

                                Close     Volume       ATR       SMA_7  \
Date                                                                     
2019-11-29 05:00:00+00:00   64.773544   46617600       NaN         NaN   
2019-12-02 05:00:00+00:00   64.024612   94487200       NaN         NaN   
2019-12-03 05:00:00+00:00   62.883064  114430400       NaN         NaN   
2019-12-04 05:00:00+00:00   63.438072   67181600       NaN         NaN   
2019-12-05 05:00:00+00:00   64.368797   74424400       NaN         NaN   
...                               ...        ...       ...         ...   
2024-11-19 05:00:00+00:00  228.279999   36211800  3.920793  225.976523   
2024-11-20 05:00:00+00:00  229.000000   35169600  3.937822  226.628999   
2024-11-21 05:00:00+00:00  228.520004   42108300  4.010990  226.758000   
2024-11-22 05:00:00+00:00  229.869995   38168300  3.817992  227.048999   
2024-11-25 05:00:00+00:00  232.869995   90152800  3.773992  227.912999   

                               SMA_30

In [33]:
def security_combine(df_main, df1, df2):
    """
    Combining the data from other indexes to be used as variabels
    Args:
        df_main (dataframe): Main secuirty which will have the others cols added
        df_1 (dataframe): Sub index which will be added to df_main, will be used for S&P 500
        df_2 (dataframe): Sub index which will be added to df_main, will be used for VIx 
    Returns:
        df_mained: Combined df_main with df1 & df2
    """

    df1 = df1.reindex(df_main.index, method='nearest')  # Align S&P 500 data
    df2 = df2.reindex(df_main.index, method='nearest')  # Align VIX data
    
    df_main = pd.merge(df_main, df1.rename(columns={"Close": "S&P 500 Data"}), 
                       left_index=True, right_index=True, how="outer")

    # Add VIX Data
    df_main = pd.merge(df_main, df2.rename(columns={"Close": "VIX"}), 
                       left_index=True, right_index=True, how="outer")
    
    return df_main

In [34]:
df_combined = security_combine(dis_refined, SP500_data, VIX_data)
df_combined

,Close,Volume,ATR,SMA_7,SMA_30,OBV,S&P 500 Data,VIX
Date,,,,,,,,
2019-11-29 05:00:00+00:00,64.773544,46617600,NaN,NaN,NaN,4.661760e+07,3140.979980,12.620000
2019-12-02 05:00:00+00:00,64.024612,94487200,NaN,NaN,NaN,-4.786960e+07,3113.870117,14.910000
2019-12-03 05:00:00+00:00,62.883064,114430400,NaN,NaN,NaN,-1.623000e+08,3093.199951,15.960000
2019-12-04 05:00:00+00:00,63.438072,67181600,NaN,NaN,NaN,-9.511840e+07,3112.760010,14.800000
2019-12-05 05:00:00+00:00,64.368797,74424400,NaN,NaN,NaN,-2.069400e+07,3117.429932,14.520000
...,...,...,...,...,...,...,...,...
2024-11-19 05:00:00+00:00,228.279999,36211800,3.920793,225.976523,227.134734,3.116585e+09,5916.979980,16.350000
2024-11-20 05:00:00+00:00,229.000000,35169600,3.937822,226.628999,227.266429,3.151755e+09,5917.109863,17.160000
2024-11-21 05:00:00+00:00,228.520004,42108300,4.010990,226.758000,227.386325,3.109647e+09,5948.709961,16.870001


In [35]:
def line_of_best_fit(X, y):
    """ 
    Finds line of best fit based off of a set of vectors
    Args:
        X (array): can be either 1-d or 2-d
        Y (array): can be either 1-d or 2-d
    Returns:
        p (array): 1d array, giving line and slope
    """

    if X.ndim == 1:
        X = add_bias_column(X).T
    else:
        X = X.T
    XTXinv = np.linalg.inv(np.matmul(X, X.T))
    p = np.matmul(XTXinv, np.matmul(X, y))
    return p

    

Regressions: